In [4]:
import dspy as ds
import pandas as pd
import numpy as np
from rich import print
from dspy.datasets.gsm8k import GSM8K, gsm8k_metric
from tqdm import tqdm
from dspy.evaluate import Evaluate
from dspy.teleprompt import BootstrapFewShot
from dspy.datasets import HotPotQA

tqdm.pandas()

%load_ext rich


The rich extension is already loaded. To reload it, use:
  %reload_ext rich


## GSM8K


In [3]:
turbo = ds.OpenAI(model="gpt-3.5-turbo-instruct", max_tokens=250)
ds.settings.configure(lm=turbo, log_openai_usage=True)

In [17]:
gsm8k = GSM8K()

train_gsm8k, dev_gsm8k = gsm8k.train[:10], gsm8k.dev[:10]

print(train_gsm8k)


100%|██████████| 1319/1319 [00:00<00:00, 28036.56it/s]


[
    Example({'question': "The result from the 40-item Statistics exam Marion and Ella took already came out. Ella 
got 4 incorrect answers while Marion got 6 more than half the score of Ella. What is Marion's score?", 
'gold_reasoning': "Ella's score is 40 items - 4 items = <<40-4=36>>36 items. Half of Ella's score is 36 items / 2 =
<<36/2=18>>18 items. So, Marion's score is 18 items + 6 items = <<18+6=24>>24 items.", 'answer': '24'}) 
(input_keys={'question'}),
    Example({'question': "Stephen made 10 round trips up and down a 40,000 foot tall mountain. If he reached 3/4 of
the mountain's height on each of his trips, calculate the total distance he covered.", 'gold_reasoning': 'Up a 
mountain, Stephen covered 3/4*40000 = <<3/4*40000=30000>>30000 feet. Coming down, Stephen covered another 30000 
feet, making the total distance covered in one round to be 30000+30000 = <<30000+30000=60000>>60000. Since Stephen 
made 10 round trips up and down the mountain, he covered 10*60000 = <<10*60000=600000>>600000', 'answer': 
'600000'}) (input_keys={'question'}),
    Example({'question': 'Bridget counted 14 shooting stars in the night sky.  Reginald counted two fewer shooting 
stars than did Bridget, but Sam counted four more shooting stars than did Reginald.  How many more shooting stars 
did Sam count in the night sky than was the average number of shooting stars observed for the three of them?', 
'gold_reasoning': 'Reginald counted two fewer shooting stars than did Bridget, or a total of 14-2=<<14-2=12>>12 
shooting stars. Sam counted 4 more shooting stars than did Reginald, or a total of 12+4=16 shooting stars. The 
average number of shooting stars observed for the three of them was (14+12+16)/3 = <<14=14>>14 shooting stars. 
Thus, Sam counted 16-14=2 more shooting stars than was the average number of shooting stars observed for the three 
of them.', 'answer': '2'}) (input_keys={'question'}),
    Example({'question': 'Sarah buys 20 pencils on Monday. Then she buys 18 more pencils on Tuesday. On Wednesday 
she buys triple the number of pencils she did on Tuesday. How many pencils does she have?', 'gold_reasoning': 'By 
adding together Monday and Tuesday, Saah has 20+18= <<20+18=38>>38 pencils On Wednesday, she buys 3 * 18= 
<<3*18=54>>54 pencils All together, Sarah has 38+54= <<38+54=92>>92 pencils', 'answer': '92'}) 
(input_keys={'question'}),
    Example({'question': 'Rookie police officers have to buy duty shoes at the full price of $85, but officers who 
have served at least a year get a 20% discount. Officers who have served at least three years get an additional 25%
off the discounted price. How much does an officer who has served at least three years have to pay for shoes?', 
'gold_reasoning': 'Cops that served a year pay $85 * 0.2 = $<<85*0.2=17>>17 less. Cops that served a year pay $85 -
$17 = $<<85-17=68>>68. Cops that served at least 3 years get a $68 * 0.25 = $<<68*0.25=17>>17 discount. Cops that 
served at least 3 years pay $68 - $17 = $<<68-17=51>>51 for shoes.', 'answer': '51'}) (input_keys={'question'}),
    Example({'question': "The average score on last week's Spanish test was 90.  Marco scored 10% less than the 
average test score and Margaret received 5 more points than Marco.  What score did Margaret receive on her test?", 
'gold_reasoning': 'The average test score was 90 and Marco scored 10% less so 90*.10 = <<90*.10=9>>9 points lower 
The average test score was 90 and Marco scored 9 points less so his test score was 90-9 = <<90-9=81>>81 Margret 
received 5 more points than Marco whose test score was 81 so she made 5+81 = <<5+81=86>>86 on her test', 'answer': 
'86'}) (input_keys={'question'}),
    Example({'question': 'A third of the contestants at a singing competition are female, and the rest are male. If
there are 18 contestants in total, how many of them are male?', 'gold_reasoning': 'There are 18/3 = <<18/3=6>>6 
female contestants. There are 18-6 = <<18-6=12>>12 male contestants.', 'answer': '12'}) (input

In [28]:
print(train_gsm8k[0])

Example({'question': "The result from the 40-item Statistics exam Marion and Ella took already came out. Ella got 4
incorrect answers while Marion got 6 more than half the score of Ella. What is Marion's score?", 'gold_reasoning': 
"Ella's score is 40 items - 4 items = <<40-4=36>>36 items. Half of Ella's score is 36 items / 2 = <<36/2=18>>18 
items. So, Marion's score is 18 items + 6 items = <<18+6=24>>24 items.", 'answer': '24'}) (input_keys={'question'})

### Define the modules


In [33]:
class CoT(ds.Module):
    def __init__(self):
        super().__init__()
        self.prog = ds.ChainOfThought("question -> answer")

    def forward(self, question):
        return self.prog(question=question)

In [43]:
teleprompter = BootstrapFewShot(
    metric=gsm8k_metric, max_bootstrapped_demos=4, max_labeled_demos=4
)

optimized_cot = teleprompter.compile(CoT(), trainset=train_gsm8k, valset=dev_gsm8k)


 50%|█████     | 5/10 [00:08<00:08,  1.67s/it]


Bootstrapped 4 full traces after 6 examples in round 0.



prog = ChainOfThought(StringSignature(question -> answer
    instructions='Given the fields `question`, produce the fields `answer`.'
    question = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'input', 'prefix': 'Question:', 'desc': '${question}'})
    answer = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'output', 'prefix': 'Answer:', 'desc': '${answer}'})
))

In [52]:
optimized_cot(question="What is the capital of France?")


Prediction(
    rationale='find the answer. First, we know that the country is France. Then, we know that the capital is the most important city in a country. Therefore, the capital of France is Paris.',
    answer='Paris'
)

In [53]:
evalute = Evaluate(
    devset=dev_gsm8k,
    metric=gsm8k_metric,
    num_threads=4,
    display_progress=True,
    display_table=1,
)

In [54]:
evalute(optimized_cot)

Average Metric: 9 / 10  (90.0): 100%|██████████| 10/10 [00:04<00:00,  2.46it/s]


Average Metric: 9 / 10  (90.0%)


e:\Projects\langchain_rag\venv\lib\site-packages\dspy\evaluate\evaluate.py:266: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['✔️ [True]']' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  df.loc[:, metric_name] = df[metric_name].apply(


,question,gold_reasoning,example_answer,rationale,pred_answer,gsm8k_metric
0,"20 birds migrate on a seasonal basis from one lake to another, searching for food. If they fly from lake Jim to lake Disney in...",The birds' flying distance between Lake Jim through lake Disney to lake London is 50+60 = <<50+60=110>>110 miles. Since each bird flies the 110 miles...,2200,"calculate the combined distance traveled by the birds. First, we know that the birds flew 50 miles from lake Jim to lake Disney and then...",2200 miles,✔️ [True]


<IPython.core.display.HTML object>

90.0

In [60]:
turbo.inspect_history()




Given the fields `question`, produce the fields `answer`.

---

Follow the following format.

Question: ${question}
Reasoning: Let's think step by step in order to ${produce the answer}. We ...
Answer: ${answer}

---

Question: The result from the 40-item Statistics exam Marion and Ella took already came out. Ella got 4 incorrect answers while Marion got 6 more than half the score of Ella. What is Marion's score?
Reasoning: Let's think step by step in order to find Marion's score. We know that Ella got 4 incorrect answers, which means she got 36 correct answers out of 40. We also know that Marion got 6 more than half of Ella's score, which is 6 more than 36/2 = 18. Therefore, Marion's score is 18 + 6 = 24.
Answer: 24

---

Question: Bridget counted 14 shooting stars in the night sky. Reginald counted two fewer shooting stars than did Bridget, but Sam counted four more shooting stars than did Reginald. How many more shooting stars did Sam count in the night sky than was the average n

"\n\n\nGiven the fields `question`, produce the fields `answer`.\n\n---\n\nFollow the following format.\n\nQuestion: ${question}\nReasoning: Let's think step by step in order to ${produce the answer}. We ...\nAnswer: ${answer}\n\n---\n\nQuestion: The result from the 40-item Statistics exam Marion and Ella took already came out. Ella got 4 incorrect answers while Marion got 6 more than half the score of Ella. What is Marion's score?\nReasoning: Let's think step by step in order to find Marion's score. We know that Ella got 4 incorrect answers, which means she got 36 correct answers out of 40. We also know that Marion got 6 more than half of Ella's score, which is 6 more than 36/2 = 18. Therefore, Marion's score is 18 + 6 = 24.\nAnswer: 24\n\n---\n\nQuestion: Bridget counted 14 shooting stars in the night sky. Reginald counted two fewer shooting stars than did Bridget, but Sam counted four more shooting stars than did Reginald. How many more shooting stars did Sam count in the night sky 

## With retrieval module


In [5]:
turbo = ds.OpenAI(model="gpt-3.5-turbo")
colbertv2_wiki17_abstracts = ds.ColBERTv2(
    url="http://20.102.90.50:2017/wiki17_abstracts"
)

ds.settings.configure(lm=turbo, rm=colbertv2_wiki17_abstracts, log_openai_usage=True)


In [6]:
dataset_hpqa = HotPotQA(
    train_seed=1, train_size=20, eval_seed=2023, dev_size=50, test_size=0
)

e:\Projects\langchain_rag\venv\lib\site-packages\datasets\table.py:1421: FutureWarning: promote has been superseded by promote_options='default'.
  table = cls._concat_blocks(blocks, axis=0)


In [24]:
trainset_hpqa = [x.with_inputs("question") for x in dataset_hpqa.train]
devset_hpqa = [x.with_inputs("question") for x in dataset_hpqa.dev]

len(trainset_hpqa), len(devset_hpqa)


(20, 50)

In [25]:
print(trainset_hpqa[2])

Example({'question': 'Which of these publications was most recently published, Who Put the Bomp or Self?', 
'answer': 'Self'}) (input_keys={'question'})

In [26]:
print(devset_hpqa[0])

Example({'question': 'Are both Cangzhou and Qionghai in the Hebei province of China?', 'answer': 'no', 
'gold_titles': {'Cangzhou', 'Qionghai'}}) (input_keys={'question'})

### Signature

In [27]:
class BasicQA(ds.Signature):
    """Answer questions with short factoid layers"""

    question = ds.InputField()
    answer = ds.OutputField(desc='often between 1 and 5 words')

In [28]:
BasicQA


BasicQA(question -> answer
    instructions='Answer questions with short factoid layers'
    question = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'input', 'prefix': 'Question:', 'desc': '${question}'})
    answer = Field(annotation=str required=True json_schema_extra={'desc': 'often between 1 and 5 words', '__dspy_field_type': 'output', 'prefix': 'Answer:'})
)

In [29]:
generate_answer = ds.Predict(BasicQA)

In [41]:
pred = generate_answer(question=devset_hpqa[3].question)

In [42]:
print(f'Question: {devset_hpqa[3].question}')
print(f'Answer: {pred.answer}')

Question: What river is near the Crichton Collegiate Church?

Answer: River Nith

In [43]:
turbo.inspect_history(n=3)




Answer questions with short factoid layers

---

Follow the following format.

Question: ${question}
Answer: often between 1 and 5 words

---

Question: What river is near the Crichton Collegiate Church?
Answer: River Nith





'\n\n\nAnswer questions with short factoid layers\n\n---\n\nFollow the following format.\n\nQuestion: ${question}\nAnswer: often between 1 and 5 words\n\n---\n\nQuestion: What river is near the Crichton Collegiate Church?\nAnswer:\x1b[32m River Nith\x1b[0m\n\n\n'

In [49]:
generate_cot = ds.ChainOfThought(BasicQA)

pred = generate_cot(question=devset_hpqa[14].question)

print(f'Question: {devset_hpqa[14].question}')
print(f'Thought: {pred.rationale.split(".", 1)[1].strip()}')
print(f'Answer: {pred.answer}')


Question: Lord North Street has a resident in which former Conservative MP who received an 18-month prison sentence
for perjury in 1999?

Thought: We know that Lord North Street is a prestigious address in London, and the former Conservative MP who 
received an 18-month prison sentence for perjury in 1999 is Jonathan Aitken.

Answer: Jonathan Aitken

In [62]:
retrieve = ds.Retrieve(k=3)

top_k_passages = retrieve(devset_hpqa[14].question).passages

print(
    f"Top {retrieve.k} passages for question: {devset_hpqa[14].question} \n",
    "-" * 30,
    "\n",
)

for idx, passage in enumerate(top_k_passages):
    print(f"{idx+1}]", passage, "\n")


Top 3 passages for question: Lord North Street has a resident in which former Conservative MP who received an 
18-month prison sentence for perjury in 1999? 
 ------------------------------

1] Jonathan Aitken | Jonathan William Patrick Aitken (born 30 August 1942) is a former Conservative Member of 
Parliament in the United Kingdom (1974–97), and a former Cabinet minister. He was convicted of perjury in 1999 and 
received an 18-month prison sentence, of which he served seven months. He is currently president of Christian 
Solidarity Worldwide. Aitken was also a member of the Privy Council of the United Kingdom.

2] David Chaytor | David Michael Chaytor (born 3 August 1949) is a former British Labour Party politician, who was 
the Member of Parliament (MP) for Bury North from 1997 to 2010. He was the first member of Parliament to be 
sentenced following the United Kingdom Parliamentary expenses scandal of 2009.

3] Elliot Morley | Elliot Anthony Morley (born 6 July 1952) is a British former Labour Party politician, who was 
the Member of Parliament for Glanford and Scunthorpe from 1987 to 1997 and then Scunthorpe from 1997 to 2010. In 
2009, he was accused by "The Daily Telegraph" of continuing to claim parliamentary expenses for a mortgage that had
already been repaid. Morley was prosecuted and on 7 April 2011 pleaded guilty in Southwark Crown Court to two 
counts of false accounting, involving over £30,000. On 20 May 2011, he was sentenced to 16 months' imprisonment. He
was released from prison on 20 September 2011 having served a quarter of his sentence.

In [63]:
retrieve("Who is the president of the United States?").passages


[
    'President of the United States (disambiguation) | The President of the United States (of America) has been chief of the United States executive branch since 1789. The current U.S. President is Donald Trump.',
    'President of the United States | The President of the United States (informally referred to as "POTUS") is the head of state and head of government of the United States. The president directs the executive branch of the federal government and is the commander-in-chief of the United States Armed Forces.',
    'List of Presidents of the United States | The President of the United States is the elected head of state and head of government of the United States. The president leads the executive branch of the federal government and is the commander-in-chief of the United States Armed Forces. The president is indirectly elected to a four-year term by the people through the Electoral College (or by the House of Representatives, should the Electoral College fail to award an a